## Persona pattern and reasoning

* "act as persona, whenever you produce an output, produce the outputs that persona would produce"
* access to expertise encapsulated within the LLM, ie it is token efficient
* expertise (act as data scientist), attitude (act as sceptic), tool or object (act as linux terminal), character
* act as persona x, perform task y

### Simple multi-agent system with personas

* main agent calling tool, with persona imprinted (other agent)
* main agent processing output of the other agent
* boundary between agents needs to be set-up, ie different personas

#### Prompting for Expertise with the Persona Pattern

When building agents that can leverage specialized expertise, we can have the LLM adopt expert personas using the **Persona Pattern**. The `prompt_expert` tool serves as the foundation, enabling interactions with experts who have specific backgrounds and perspectives. Below, we build specialized expert tools for a software development workflow.

##### Base Expert Consultation Tool

```python
@register_tool()
def prompt_expert(action_context: ActionContext, description_of_expert: str, prompt: str) -> str:
    """
    Generate a response from an expert persona.
    
    The expert's background and specialization should be thoroughly described to ensure
    responses align with their expertise. The prompt should be focused on topics within
    their domain of knowledge.
    
    Args:
        description_of_expert: Detailed description of the expert's background and expertise
        prompt: The specific question or task for the expert
        
    Returns:
        The expert's response
    """
    generate_response = action_context.get("llm")
    response = generate_response(Prompt(messages=[
        {"role": "system", 
         "content": f"Act as the following expert and respond accordingly: {description_of_expert}"},
        {"role": "user", "content": prompt}
    ]))
    return response
```

##### Dynamic Experts via the Persona Pattern

While pre-defined expert tools are useful, a major strength of this pattern is **on-the-fly creation of new experts** tailored to the challenge at hand. Instead of relying only on generalists, the agent can instantiate a specialist persona (e.g., “ML pipeline optimization on specialized hardware”) whenever needed.

##### Prebuilt Experts

Each tool below wraps `prompt_expert` with a focused persona and a targeted prompt.

###### Technical Documentation Expert

```python
@register_tool(tags=["documentation"])
def generate_technical_documentation(action_context: ActionContext, code_or_feature: str) -> str:
    """
    Generate technical documentation by consulting a senior technical writer.
    This expert focuses on creating clear, comprehensive documentation for developers.
    
    Args:
        code_or_feature: The code or feature to document
    """
    return prompt_expert(
        action_context=action_context,
        description_of_expert="""
        You are a senior technical writer with 15 years of experience in software documentation.
        You have particular expertise in:
        - Writing clear and precise API documentation
        - Explaining complex technical concepts to developers
        - Documenting implementation details and integration points
        - Creating code examples that illustrate key concepts
        - Identifying and documenting important caveats and edge cases
        
        Your documentation is known for striking the perfect balance between completeness
        and clarity. You understand that good technical documentation serves as both
        a reference and a learning tool.
        """,
        prompt=f"""
        Please create comprehensive technical documentation for the following code or feature:

        {code_or_feature}

        Your documentation should include:
        1. A clear overview of the feature's purpose and functionality
        2. Detailed explanation of the implementation approach
        3. Key interfaces and integration points
        4. Usage examples with code snippets
        5. Important considerations and edge cases
        6. Performance implications if relevant
        
        Focus on providing information that developers need to effectively understand
        and work with this code.
        """
    )
```

###### QA / Test Design Expert

```python
@register_tool(tags=["testing"])
def design_test_suite(action_context: ActionContext, feature_description: str) -> str:
    """
    Design a comprehensive test suite by consulting a senior QA engineer.
    This expert focuses on creating thorough test coverage with attention to edge cases.
    
    Args:
        feature_description: Description of the feature to test
    """
    return prompt_expert(
        action_context=action_context,
        description_of_expert="""
        You are a senior QA engineer with 12 years of experience in test design and automation.
        Your expertise includes:
        - Comprehensive test strategy development
        - Unit, integration, and end-to-end testing
        - Performance and stress testing
        - Security testing considerations
        - Test automation best practices
        
        You are particularly skilled at identifying edge cases and potential failure modes
        that others might miss. Your test suites are known for their thoroughness and
        their ability to catch issues early in the development cycle.
        """,
        prompt=f"""
        Please design a comprehensive test suite for the following feature:

        {feature_description}

        Your test design should cover:
        1. Unit tests for individual components
        2. Integration tests for component interactions
        3. End-to-end tests for critical user paths
        4. Performance test scenarios if relevant
        5. Edge cases and error conditions
        6. Test data requirements
        
        For each test category, provide:
        - Specific test scenarios
        - Expected outcomes
        - Important edge cases to consider
        - Potential testing challenges
        """
    )
```

###### Code Review / Architecture Expert

```python
@register_tool(tags=["code_quality"])
def perform_code_review(action_context: ActionContext, code: str) -> str:
    """
    Review code and suggest improvements by consulting a senior software architect.
    This expert focuses on code quality, architecture, and best practices.
    
    Args:
        code: The code to review
    """
    return prompt_expert(
        action_context=action_context,
        description_of_expert="""
        You are a senior software architect with 20 years of experience in code review
        and software design. Your expertise includes:
        - Software architecture and design patterns
        - Code quality and maintainability
        - Performance optimization
        - Scalability considerations
        - Security best practices
        
        You have a talent for identifying subtle design issues and suggesting practical
        improvements that enhance code quality without over-engineering.
        """,
        prompt=f"""
        Please review the following code and provide detailed improvement suggestions:

        {code}

        Consider and address:
        1. Code organization and structure
        2. Potential design pattern applications
        3. Performance optimization opportunities
        4. Error handling completeness
        5. Edge case handling
        6. Maintainability concerns
        
        For each suggestion:
        - Explain the current issue
        - Provide the rationale for change
        - Suggest specific improvements
        - Note any trade-offs to consider
        """
    )
```

###### Product Marketing / Feature Announcement Expert

```python
@register_tool(tags=["communication"])
def write_feature_announcement(action_context: ActionContext, 
                               feature_details: str,
                               audience: str) -> str:
    """
    Write a feature announcement by consulting a product marketing expert.
    This expert focuses on clear communication of technical features to different audiences.
    
    Args:
        feature_details: Technical details of the feature
        audience: Target audience for the announcement (e.g., "technical", "business")
    """
    return prompt_expert(
        action_context=action_context,
        description_of_expert="""
        You are a senior product marketing manager with 12 years of experience in
        technical product communication. Your expertise includes:
        - Translating technical features into clear value propositions
        - Crafting compelling product narratives
        - Adapting messaging for different audience types
        - Building excitement while maintaining accuracy
        - Creating clear calls to action
        
        You excel at finding the perfect balance between technical accuracy and
        accessibility, ensuring your communications are both precise and engaging.
        """,
        prompt=f"""
        Please write a feature announcement for the following feature:

        {feature_details}

        This announcement is intended for a {audience} audience.

        Your announcement should include:
        1. A compelling introduction
        2. Clear explanation of the feature
        3. Key benefits and use cases
        4. Technical details (adapted to audience)
        5. Implementation requirements
        6. Next steps or call to action
        
        Ensure the tone and technical depth are appropriate for a {audience} audience.
        Focus on conveying both the value and the practical implications of this feature.
        """
    )
```

##### Advantages of the Persona Pattern

* Each expert’s background and specialization is clearly defined inside the tool.
* Prompts are crafted for specific task types, improving output quality.
* Tool docstrings clarify purpose and usage, aiding discoverability.
* Code remains maintainable, as each expert’s implementation stands alone.

##### Crafting Effective Experts and Prompts

Success hinges on:

* **Rich expert descriptions** that set perspective, scope, and methods.
* **Focused prompts** that ask for specific, actionable deliverables within that scope.

This pattern lets agents leverage specialized expertise while preserving clean separation of concerns and keeping core reasoning lightweight.


#### The Persona as an Agent Building Block

The persona pattern is a powerful, efficient abstraction in prompt engineering. Invoking a persona activates an integrated reasoning system the LLM has internalized, avoiding the need to enumerate exhaustive rules and methods.

**Why it works**

* **Implicit domain knowledge:** The model knows what experts in a domain prioritize and how they think.
* **Methodological understanding:** It brings dynamic procedures, not just static facts.
* **Contextual adaptation:** Approaches flex to the situation.
* **Integrated value systems:** Coherent priorities guide decisions.

**System design implications**

* **Prompt economy:** Complex reasoning in few tokens.
* **Emergent capabilities:** Useful behaviors appear without explicit enumeration.
* **Natural reasoning:** Feels like how humans/experts think.
* **Evolving understanding:** Improves as models are retrained.

**How to maximize benefits**

* **Be specific but concise:** Activate the right frame without over-constraining.
* **Leverage established roles:** Use recognizable expert titles/domains.
* **Enhance, don’t override:** Add instructions to focus the persona.
* **Test activation:** Probe early with quick questions to verify the mindset.

---

#### Expertise as a Modular Resource

Treat expertise as modular, not monolithic. Keep the core agent lean; load domain knowledge via personas with clear interfaces.

**Practical implications**

* **Knowledge management:** Update a persona when best practices evolve.
* **Resource efficiency:** Load only what’s needed per task.
* **Specialization with coordination:** Combine multiple personas for cross-domain problems.
* **Explicit boundaries:** Clarify what each persona covers.

**Implementation guidance**

* Define clear domain boundaries.
* Include factual *and* procedural knowledge.
* Design interfaces between personas (how outputs flow).
* Keep descriptions focused yet self-contained.

---

#### Meta-Cognition Through Role-Switching

Switching personas gives deliberate perspective shifts and bias checks.

**Benefits**

* **Counter biases:** Security vs. performance vs. UX lenses.
* **Manage dissonance:** Explore priorities separately, then synthesize.
* **Leverage patterns:** Use domain-specific ways of thinking.
* **Perspective-taking:** Systematically analyze from multiple roles.

**Example workflow**

1. Security persona reviews vulnerabilities.
2. UX persona ensures usability.
3. Performance persona checks scalability.
4. Agent synthesizes recommendations.

---

#### Dynamic vs. Static Expertise

**Static (pre-defined persona tools)**

* Consistent behavior, easier orchestration, testable and optimizable.

**Dynamic (on-the-fly personas)**

* Unlimited specialization, adapts to novel cases, supports exploratory work, enables iterative refinement.

**Dynamic persona helper (example)**

```python
@register_tool()
def create_and_consult_expert(action_context: ActionContext, 
                              expertise_domain: str,
                              problem_description: str) -> str:
    """
    Dynamically create and consult an expert persona for a specific domain/problem.
    """
    generate_response = action_context.get("llm")

    persona_description = generate_response(Prompt(messages=[{
        "role": "user",
        "content": f"""
        Create a detailed description of an expert in {expertise_domain} for this problem:
        {problem_description}
        Include background, specializations, problem-solving approach, and unique perspective.
        """
    }]))

    consultation_prompt = generate_response(Prompt(messages=[{
        "role": "user",
        "content": f"""
        Create a consultation prompt for an expert in {expertise_domain} addressing:
        {problem_description}
        Guide toward comprehensive, actionable recommendations.
        """
    }]))

    return prompt_expert(
        action_context=action_context,
        description_of_expert=persona_description,
        prompt=consultation_prompt
    )
```

---

#### Focused Context Windows

Persona consultations create dedicated context windows optimized for a domain.

**Effects**

* **Prioritize domain context** for relevance.
* **Manage conflicting priorities** (e.g., security vs. UX).
* **Preserve specialized vocabulary.**
* **Deepen reasoning** within limited context space.

**Crafting focused windows**

* **Persona descriptions:** Boundaries, core knowledge, perspective.
* **Consultation prompts:** Target the needed expertise and aspects.

---

#### Expertise as Documentation

Persona descriptions double as living documentation.

**Benefits**

* **Knowledge capture:** Facts + priorities + methods.
* **Onboarding aid:** Quick map of domains covered.
* **Capability mapping:** Shows what the system can/can’t do.
* **Upgrade path:** Update personas as domains evolve.

**Documentation tips**

* Be explicit about boundaries.
* Include methodology and key concepts.
* Reference standards/best practices.

---

#### Chain of Expertise

Link personas into workflows where one output feeds the next.

**Enables**

* **Progressive refinement** across stages.
* **Cross-domain integration** for complex problems.
* **Stage specialization** (design → build → test → doc).
* **Checks and balances** via mutual review.

**Implementation considerations**

* Ensure sufficient information transfer.
* Preserve critical context across steps.
* Add feedback loops and conflict resolution.

**Simplified chain (example)**

```python
def develop_feature(action_context: ActionContext, feature_request: str) -> dict:
    requirements = prompt_expert(action_context, "product manager expert",
                                 f"Turn this request into detailed requirements: {feature_request}")

    architecture = prompt_expert(action_context, "software architect expert",
                                 f"Design an architecture for these requirements: {requirements}")

    implementation = prompt_expert(action_context, "senior developer expert",
                                   f"Implement code for this architecture: {architecture}")

    tests = prompt_expert(action_context, "QA engineer expert",
                          f"Create test cases for this implementation: {implementation}")

    documentation = prompt_expert(action_context, "technical writer expert",
                                  f"Document this implementation: {implementation}")

    return {
        "requirements": requirements,
        "architecture": architecture,
        "implementation": implementation,
        "tests": tests,
        "documentation": documentation
    }
```

---

#### Knowledge Curation

Personas curate *how* knowledge is structured and applied, not just *what* is known.

**Curation elements**

* Selection of relevant knowledge.
* Contextual framing within domain models.
* Methodology definition and priorities.
* Heuristics and pitfalls.

**Best practices**

* Balance breadth and depth.
* Cover theory and practice.
* Document mental models.
* Note common mistakes.
* Update regularly.

---

#### Behavioral Consistency

Using the same persona for a domain yields predictable, reliable behavior.

**Why it matters**

* Builds user trust.
* Enables tighter system integration.
* Simplifies quality evaluation.
* Supports user learning transfer.

**How to maintain it**

* Define core principles per persona.
* Allow contextual adaptation within those principles.
* Version persona definitions for major changes.
* Monitor outputs for consistency.
